## Parameters setup

In [ ]:
dbutils.widgets.text("source_schema","ds_goblet")
dbutils.widgets.text("target_schema","nextlevel_dm")
dbutils.widgets.text("target_table","user_tab_columns")

In [ ]:
w_source_schema = dbutils.widgets.get("source_schema")
w_target_schema = dbutils.widgets.get("target_schema")
w_user_tab_columns = dbutils.widgets.get("target_table")

### Temporary table

In [ ]:
spark.sql(f"create or replace table {w_target_schema}.dual select 1 as id ")

## Defining user_tab_columns table structure

In [ ]:
user_tab_columns = spark.sql(f"select 's' as schema, 'a' as table_name, 'b' as column_name, 'c' as data_type, 'd' as ordinal_position, 'e' as comment, current_timestamp() as latest_metadata_refresh_date from {w_target_schema}.dual where 1 = 2")

## Collect and store metadata information
- listing table
- listing columns per table
- overwriting existing table and inserting new metadata

In [ ]:
tables = spark.sql(f"show tables in {w_source_schema}")
tables = tables.filter(f"istemporary = False and tableName <> '{w_user_tab_columns}'")

In [ ]:
for row in tables.collect():    
    df_table = spark.sql(f"select '{w_source_schema}' as schema, '{row.tableName}' as table_name" )
    df_table.createOrReplaceTempView(f"{w_target_schema}.table_schema")
    
    table_columns = spark.sql(f"describe table {w_source_schema}.{row.tableName}") # table describe           
    table_columns.createOrReplaceTempView(f"{w_target_schema}.table_columns")
    
    one_table_data = spark.sql("""select ts.schema, ts.table_name, lower(tc.col_name) as column_name, tc.data_type, row_number() over(order by (select null)) as ordinal_position, tc.comment, current_timestamp() as latest_metadata_refresh_date from table_columns tc
                  inner join table_schema ts on (1 = 1) """)
       
    user_tab_columns = user_tab_columns.unionAll(one_table_data)

In [ ]:
user_tab_columns.write.mode("overwrite").saveAsTable(f"{w_target_schema}.{w_user_tab_columns}")

### Dropping temporary table

In [ ]:
spark.sql(f"drop table {w_target_schema}.dual ")

## Debug

In [ ]:
%sql

select * from nextlevel_dm.user_tab_columns